# [GALAXY] - Image Reducer

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">stenv</span> environment.
</div>

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path

# Astropy Colab Imports
from astropy.io import fits
from drizzlepac.tweakreg import TweakReg
from drizzlepac.astrodrizzle import AstroDrizzle

## Notebook Setup

In [ ]:
# Get the Notebook's Directory and ChangeDir to it
NOTEBOOK_DIR = Path(__file__).resolve().parent
os.chdir(NOTEBOOK_DIR)

In [ ]:
# Data Directory
DATA_DIR = Path('mastDownload/HST')

# FLC Glob Pattern
FLC_CR_GLOB_PAT = '**/*_crclean_fl?.fits'

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in DATA_DIR.rglob(FLC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)
fileNameDict

## Align Images to GAIA

Sometimes, the FLCs have so many CRs and so few Milky Way stars, it makes aligning the images difficult.
In this case, a different strategy is to align the Hubble Pipeline DRCs/DRZs to GAIA using `TweakReg` then
assigning the found WCS solution to the associated FLCs/FLTs using `TweakBack`.

For an example of this strategy, consider the work documented in the [Abell 1367](https://github.com/wwaldron/a1367)
repo on GitHub where the [Image Reducer Notebook](https://github.com/wwaldron/a1367/blob/main/Images/ImageReducer.ipynb)
implements this methodology.

### TweakReg Data Quality Flags

This cell defines the [ACS DQ flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions) we
want to ignore in the TweakReg process. The DQ flags that are most often used are:

* [ACS DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)
* [WFC3-UVIS DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-2-uvis-data-calibration-steps#id-3.2UVISDataCalibrationSteps-3.2.3DataQualityArrayInitialization)
* [WFC3-IR DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps#id-3.3IRDataCalibrationSteps-3.3.1DataQualityInitialization)

For an example of how to implement multiple DQ flags, consider the
[Image Reducer for ESO 137-001](https://github.com/wwaldron/ESO-137-001/blob/main/Images/ImageReducer.ipynb).

<div class="alert alert-block alert-info">
    <b>Note:</b> In the cells below where <span style="font-family: 'Ariel', monospace;">TweakReg</span> is called,
    the user <i>must</i> update the <span style="font-family: 'Ariel', monospace;">updatehdr</span>
    keyword to <span style="font-family: 'Ariel', monospace;">True</span> and rerun the cell once a valid
    WCS solution is found.
    If the value is left as <span style="font-family: 'Ariel', monospace;">False</span>, the header in the
    input file will not be updated.
</div>

In [ ]:
# DQ Bits
DQ_FILLED   = 2
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX = ~(DQ_FILLED + DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX) # Ignore these

In [ ]:
# Loop through Filters
for filter, fileList in fileNameDict.items():

    # Image Dep Search Parameters
    threshold  = 6.0
    conv_width = 3.5

    # Image Find Parameters
    imagefindcfg = dict(
        # peakmax=900,
        threshold=threshold,
        conv_width=conv_width,
        dqbits=DQ_GOOD_PIX
    )

    # Align the Images to the GAIA data
    TweakReg(
        fileList,
        updatehdr=False,
        wcsname='GAIA',
        clean=True,
        configobj=None,
        refcat='../Data/GAIA/[GALAXY_SHORT]-GAIA-RefCatalog-icrs.txt',
        runfile=f'TweakReg-GAIA-{filter}.log',
        searchrad=0.35,
        minobj=15,
        tolerance=1.00,
        imagefindcfg=imagefindcfg,
        interactive=False
    )

In [ ]:
%%bash
# Move Log Files
mkdir -p logs/tweakreg
mv *.log logs/tweakreg

# Remove Intermediate Files
rm *.coo *.png

## Drizzle Images for CR Correction

Although there will be additional notes added later, it is worth noting that according to
[STScI](https://hst-docs.stsci.edu/drizzpac/chapter-6-reprocessing-with-the-drizzlepac-package/6-3-running-astrodrizzle#id-6.3RunningAstroDrizzle-SelectingtheOptimalScaleandPixfrac):

1. For sub-pixel dithered data, select an output scale that's smaller than the native scale.
It will even help in the cosmic ray rejection step.
1. A smaller final_pixfrac gives higher resolution and lower correlated noise, but also reduces
sensitivity to low-surface brightness features (though it is possible to convolve a high resolution
image later to go after low surface brightness features).
1. Keep the standard deviation of the weight map over the main part of the image to above ~0.3 of
the mean to insure that one does not lose significant signal-to-noise in ignoring the weight map in
final photometry.

To summarize the last step, a `final_scale`/`final_pixfrac` combo should be chosen such that,
for the weight image,
\begin{equation}
    \mathrm{std} \gtrsim 0.3 \, \mathrm{mean}
\end{equation}

### AstroDrizzle ACS Data Quality Flags

* [ACS DQ Flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)
* [WFC3-UVIS DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-2-uvis-data-calibration-steps#id-3.2UVISDataCalibrationSteps-3.2.3DataQualityArrayInitialization)
* [WFC3-IR DQ Flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps#id-3.3IRDataCalibrationSteps-3.3.1DataQualityInitialization)

In [ ]:
# DQ Bits
DQ_WARM_PIX = 64
DQ_BAD_COL  = 128
DQ_FULL_WELL= 256
DQ_SINK_PIX = 1024
DQ_GOOD_PIX = DQ_WARM_PIX + DQ_BAD_COL + DQ_FULL_WELL + DQ_SINK_PIX # Make these OK

### Drizzle F814W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F814W'],
    output='[GALAXY_SHORT]-F814W',
    runfile='F814W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.8,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_rot=0,
    final_scale=0.03
)

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F475W'],
    output='[GALAXY_SHORT]-F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=0.8,
    final_bits=DQ_GOOD_PIX,
    final_wcs=True,
    final_refimage='[GALAXY_SHORT]-F814W_drc.fits'
)

In [ ]:
%%bash
# Move Log Files
mkdir -p logs/astrodrizzle
mv *.log logs/astrodrizzle

# Move Final Drizzled Images
mkdir -p ProcessedImages/HST/Drizzled
mv *_dr?.fits ProcessedImages/HST/Drizzled/